In [1]:
# load libraries
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import interact
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms
import skimage.exposure as exposure
from torch.utils.data import DataLoader
import shutil
from model import conv_block, encoder_block, decoder_block, UNet

In [15]:
if torch.cuda.is_available():
    print("cuda available")    
    device = torch.device("cuda")
else:
    print("not available")
    device = torch.device("cpu")

cuda available


In [17]:
model = UNet()
model = model.to(device)
model.load_state_dict(torch.load("checkpoints/checkpoint_unet2d_relu_lr=1e-4_rescale_t=0.5_adam_20_epochs_batch=16_reduce_scheduler.pth", map_location=device))

<All keys matched successfully>

In [18]:
test = "data/test/volume/"
files = os.listdir(test)

In [20]:
#os.mkdir("inferences")

In [22]:
for file in files : 
    img = nib.load(test+file)
    img_v = img.get_fdata()
    x, y, z = img_v.shape
    prediction = np.zeros((x, y, z))
    img_vol = torch.from_numpy(img_v).to(torch.float32).unsqueeze(0).to(device)
    for idx in range(z):
        x = img_vol.unsqueeze(0)[:,:,:,:,idx]
        pred = model(x)
        prediction[:,:,idx] = (torch.sigmoid(pred).squeeze(0, 1).cpu().detach().numpy()>.5)
    prediction = nib.Nifti1Image(prediction, img.affine)
    nib.save(prediction, "inferences/"+file[6:9]+".nii.gz")

In [ ]:
img = nib.load("inferences/")
img_v = img.get_fdata()

In [29]:
x = "inferencesLUNG1-038_vol.nii.gz"

In [13]:
liste = os.listdir("inferences/")
liste.sort()

In [14]:
liste

['LUNG1-001.nii.gz',
 'LUNG1-002.nii.gz',
 'LUNG1-003.nii.gz',
 'LUNG1-004.nii.gz',
 'LUNG1-005.nii.gz',
 'LUNG1-006.nii.gz',
 'LUNG1-007.nii.gz',
 'LUNG1-008.nii.gz',
 'LUNG1-009.nii.gz',
 'LUNG1-010.nii.gz',
 'LUNG1-011.nii.gz',
 'LUNG1-012.nii.gz',
 'LUNG1-013.nii.gz',
 'LUNG1-014.nii.gz',
 'LUNG1-015.nii.gz',
 'LUNG1-016.nii.gz',
 'LUNG1-017.nii.gz',
 'LUNG1-018.nii.gz',
 'LUNG1-019.nii.gz',
 'LUNG1-020.nii.gz',
 'LUNG1-021.nii.gz',
 'LUNG1-022.nii.gz',
 'LUNG1-023.nii.gz',
 'LUNG1-024.nii.gz',
 'LUNG1-025.nii.gz',
 'LUNG1-026.nii.gz',
 'LUNG1-027.nii.gz',
 'LUNG1-028.nii.gz',
 'LUNG1-029.nii.gz',
 'LUNG1-030.nii.gz',
 'LUNG1-031.nii.gz',
 'LUNG1-032.nii.gz',
 'LUNG1-033.nii.gz',
 'LUNG1-034.nii.gz',
 'LUNG1-035.nii.gz',
 'LUNG1-036.nii.gz',
 'LUNG1-037.nii.gz',
 'LUNG1-038.nii.gz',
 'LUNG1-039.nii.gz',
 'LUNG1-040.nii.gz',
 'LUNG1-041.nii.gz',
 'LUNG1-042.nii.gz',
 'LUNG1-043.nii.gz',
 'LUNG1-044.nii.gz',
 'LUNG1-045.nii.gz',
 'LUNG1-046.nii.gz',
 'LUNG1-047.nii.gz',
 'LUNG1-048.n

In [6]:
"ajsbas.aknas".split(".")

['ajsbas', 'aknas']

In [8]:
for file in liste : 
    s = file.split(".")
    i = int(s[0])
    if i<10 : 
        os.rename("inferences/"+file, "inferences/LUNG1-00"+str(i)+".nii.gz")        
    elif i<100 : 
        os.rename("inferences/"+file, "inferences/LUNG1-0"+str(i)+".nii.gz")  
    else : 
        os.rename("inferences/"+file, "inferences/LUNG1-"+str(i)+".nii.gz")        

In [11]:
os.rename("inferences/100.nii.gz", "inferences/LUNG1-100.nii.gz")        